In [5]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [6]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
TRAINING_DIR = "/home/sourjayon/dataset"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/home/sourjayon/dataset"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1376 images belonging to 2 classes.
Found 1376 images belonging to 2 classes.


In [8]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [9]:
history = model.fit(train_generator,
                              epochs=8,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/8
138/138 [==============================] - ETA: 0s - loss: 0.5979 - acc: 0.6621

2022-04-11 14:10:33.441374: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model2-001.model/assets
138/138 [==============================] - 37s 267ms/step - loss: 0.5979 - acc: 0.6621 - val_loss: 0.2876 - val_acc: 0.9433
Epoch 2/8
138/138 [==============================] - 37s 265ms/step - loss: 0.3991 - acc: 0.8619 - val_loss: 0.2052 - val_acc: 0.9462
Epoch 3/8
138/138 [==============================] - 36s 263ms/step - loss: 0.2801 - acc: 0.8997 - val_loss: 0.1806 - val_acc: 0.9273
Epoch 4/8
138/138 [==============================] - 37s 266ms/step - loss: 0.2965 - acc: 0.8794 - val_loss: 0.1135 - val_acc: 0.9680
Epoch 5/8
138/138 [==============================] - 36s 262ms/step - loss: 0.2166 - acc: 0.9164 - val_loss: 0.1040 - val_acc: 0.9702
Epoch 6/8
138/138 [==============================] - 35s 256ms/step - loss: 0.2142 - acc: 0.9172 - val_loss: 0.1449 - val_acc: 0.9586
Epoch 7/8
138/138 [==============================] - 36s 261ms/step - loss: 0.1796 - acc: 0.9302 - val_loss: 0.0746 - val_acc: 0.9724
Epoch 8/8
138

In [11]:
model.save('./model-010.h5')

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model=load_model("./model-010.h5")

labels_dict={0:'without_mask',1:'mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) 


classifier = cv2.CascadeClassifier('/home/sourjayon/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1)

    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    faces = classifier.detectMultiScale(mini)

    for f in faces:
        (x, y, w, h) = [v * size for v in f]
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    if key == 27: 
        break

webcam.release()

cv2.destroyAllWindows()